# Finding ground states

In this example, we illustrate how to search for ground states using a ground-state solver based on a cluster expansion. The implementation of will only find the ground-state in a given supercell, and so the solution represents an upper-bound to the global ground-state of a bulk crystal.

In [1]:
import copy

import numpy as np
import matplotlib.pyplot as plt
from monty.serialization import loadfn, dumpfn
from pymatgen.analysis.ewald import EwaldSummation
from pymatgen.core import Structure

from smol.cofe import ClusterSubspace, ClusterExpansion
from smol.cofe.extern import EwaldTerm
from smol.cofe.space.domain import get_allowed_species

from smol.moca import Ensemble

from smol.capp.generate.groundstate.solver import GroundStateSolver

### 0) Create a Cluster Subspace based on the disordered structure with an Ewald term

In [2]:
lmof = loadfn('data/lmof_prim.json') 

In [3]:
subspace = ClusterSubspace.from_cutoffs(
    structure=lmof, cutoffs={2: 4.0, 3: 3.0}
)
 # Add an external Ewald Term
subspace.add_external_term(EwaldTerm(eta=None))

In [4]:
print(subspace)

Basis/Orthogonal/Orthonormal : indicator/False/False
       Unit Cell Composition : Li+0.8 Mn2+0.85 Mn4+0.25 Mn3+0.25 O2-0.5 F-0.5
            Number of Orbits : 30
No. of Correlation Functions : 168
             Cluster Cutoffs : 2: 3.64, 3: 2.97
              External Terms : [EwaldTerm(total)]
Orbit Summary
 ------------------------------------------------------------------------
 |  ID     Degree    Cluster Diameter    Multiplicity    No. Functions  |
 |   0       0             NA                 0                1        |
 |   1       1            0.0000              2                2        |
 |   2       1            0.0000              1                1        |
 |   3       1            0.0000              1                4        |
 |   4       2            1.8187              8                2        |
 |   5       2            1.8187              8                8        |
 |   6       2            2.1000              6                3        |
 |   7       2        

### 1) Create random ECIs and ClusterExpansion
For this example we will use random coefficients for the ECIs as a synthetic model.

In [5]:
coefs = np.random.random(size=subspace.num_corr_functions+1)
coefs[0] = -10
coefs[-1] = 0.1

In [6]:
ce = ClusterExpansion(subspace, coefs)

/home/lbluque/Develop/pymatgen/pymatgen/core/periodic_table.py:1051: UserWarning: Use of properties is now deprecated. Set the spin by setting the spin arg instead.
  warnings.warn("Use of properties is now deprecated. Set the spin by setting the spin arg instead.")


### 2) Create a semi-grand canonical ensemble
By default, the `Ensemble` object uses `ClusterDecompositionProcessor`, which would greatly reduce the amount of many-body terms in pseudo-Boolean function. Switching to `ClusterExpansionProcessor` is an option as well but not recommended.

The supercell size is defined within the Ensemble. Here, we use a supercell containing 2 primitive units. A semigrand-canonical ensemble is defined as an Ensemble object with chemical potentials specified.

In [7]:
chempots = {
    "Li+": 0.0, "Mn2+": 0.0, "Mn3+": 0.0, "Mn4+": 0.0, "Vacancy":0.0, "O2-":0.0, "F-": 0.0
}
grand_ens = Ensemble.from_cluster_expansion(
    ce, np.diag([2, 1, 1]), chemical_potentials=chempots
)

### 3) Create a solver instance from the ensemble
Charge-balance constraints are included by default for any structure with charged species. If any other constraint is needed, refer to the documentation of `GroundStateSolver`.

The default solver is "SCIP". For other solvers supported by cvxpy, see: https://www.cvxpy.org/tutorial/advanced/index.html#setting-solver-options

In [8]:
# Use 1e-6 as a cutoff to cluster terms. Any term with coefficient lower than 1e-6 will not be included into the optimization.
grand_solver = GroundStateSolver(grand_ens, term_coefficients_cutoff=1e-6)
print("Number of variables:", grand_solver._canonicals.variables.size)
print("Number of auxiliary variables:", grand_solver._canonicals.auxiliary_variables.size)
print("Number of constraints:", len(grand_solver._canonicals.constraints))

Number of variables: 26
Number of auxiliary variables: 2997
Number of constraints: 11646


### 4) Solve the problem in semi-grand canonical ensemble

In [9]:
_, energy = grand_solver.solve()
structure = grand_solver.ground_state_structure

print("Ground-state energy, un-normalized(eV):", energy)
print("Ground-state structure:", structure)

Ground-state energy, un-normalized(eV): -27.45088000967906
Ground-state structure: Full Formula (Mn1 O2)
Reduced Formula: MnO2
abc   :   5.939700   2.969850   2.969850
angles:  60.000000  60.000000  60.000000
pbc   :       True       True       True
Sites (3)
  #  SP       a    b    c
---  ----  ----  ---  ---
  0  Mn4+  0.5   0    0
  1  O2-   0.25  0.5  0.5
  2  O2-   0.75  0.5  0.5


### 5) Create and solve a canonical ensemble problem
Our solver also supports solving the ground-state in canonical ensembles. In doing so, one only needs to create a canonical ensemble (i.e., an ensemble with no chemical potentials provided). When using a canonical ensemble, note that either a fixed composition or an intial occupancy to determine the composition to fix must be provided as an argument to initialize the solver.

In [10]:
# Creating the ensemble.
canonical_ensemble = Ensemble.from_cluster_expansion(ce, np.diag([2, 1, 1]))

In [11]:
# Fix to LiMnO2.
canonical_solver = GroundStateSolver(
    canonical_ensemble, term_coefficients_cutoff=1e-6,
    initial_occupancy=[0, 2, 2, 4 ,2 ,2 ,0, 0],
    solver="GUROBI"
)

print("Number of variables:", canonical_solver._canonicals.variables.size)
print("Number of auxiliary variables:", canonical_solver._canonicals.auxiliary_variables.size)
print("Number of constraints:", len(canonical_solver._canonicals.constraints))

Number of variables: 26
Number of auxiliary variables: 2997
Number of constraints: 11655


In [12]:
_, energy = canonical_solver.solve()
structure = canonical_solver.ground_state_structure

print("Ground-state energy, un-normalized(eV):", energy)
print("Ground-state structure:", structure)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-08
Ground-state energy, un-normalized(eV): -22.768251372269486
Ground-state structure: Full Formula (Li1 Mn1 O2)
Reduced Formula: LiMnO2
abc   :   5.939700   2.969850   2.969850
angles:  60.000000  60.000000  60.000000
pbc   :       True       True       True
Sites (4)
  #  SP        a     b     c
---  ----  -----  ----  ----
  0  Mn3+  0      0     0
  1  Li+   0.375  0.75  0.75
  2  O2-   0.25   0.5   0.5
  3  O2-   0.75   0.5   0.5
